In [1]:
%matplotlib inline
import numpy as np
import pandas as pa
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob, os
import matplotlib.style as style 
#style.available
plt.style.use('seaborn') 
#style.use('seaborn-poster') #sets the size of the charts
import pprint
from IPython.display import HTML

filled_markers = ('o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X')
dash_styles = ["",
               (4, 1.5),
               (1, 1),
               (3, 1, 1.5, 1),
               (5, 1, 1, 1),
               (5, 1, 2, 1, 2, 1),
               (2, 2, 3, 1.5),
               (1, 2.5, 3, 1.2),
              (4,2,2,1)]

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pa.options.display.max_rows = 50


In [2]:
#Clean up result directory first
results_path = 'timing_results/*'
r = glob.glob(results_path)
for i in r:
    os.remove(i)

#Then clean the plan directory
plans_path = 'query_plans/*'
r = glob.glob(plans_path)
for i in r:
    os.remove(i)
    
#Then clean the timing_results directory
timing_histogram_path = 'hist_timing_results/*'
r = glob.glob(timing_histogram_path)
for i in r:
    os.remove(i)
    
#Setup variables
result_list = []
histogram_timing_results_list = []
query_plan_list = []
i = -1

# Format the .result files from our mtr run so that we can import them using pandas csv importer

In [3]:
#with open("/home/svestrhe/Documents/Forprosjekt/code/mysql-server/mysql-test/suite/histogram_plugin/r/execute_timing_loki12.result") as openfileobject:
with open("/home/svestrhe/Documents/Forprosjekt/code/mysql-server/mysql-test/suite/histogram_plugin/r/execute_timing_local.result") as openfileobject:
    for line in openfileobject:
        if not "#" in line[:5]:
            if "Query" in line:
                i+=1
                result_list.append([line]) 
                query_plan_list.append([])
            if "Analyse statement" in line[:17]:
                histogram_timing_results_list.append([line])
            if "select" in line[:8]:
                result_list[i].append(line)
            if "EXPLAIN" in line[:7] or "->" in line or "id" in line[:4] or "1" in line[:2]:
                query_plan_list[i].append(line)
            if "explain" in line[:7]:
                query_plan_list[i].append("\n")
                query_plan_list[i].append(line)
            if "analyze table measurement" in line.lower():
                histogram_timing_results_list[i].append(line)
    i = -1   #Cleanup


In [4]:
#Export the timing results for queries
array_result = np.array(result_list)
result_list=[]      #Cleanup so that we can run the cells again and again without having to restart our kernel. 
counter = 0
for result in array_result:
    string="timing_results/{}.csv".format(counter)
    f = open(string,"w+")
    for line in result:
        f.write(line)
    f.close()
    counter+=1

    
#Export the timing results for histograms
array_hist_result = np.array(histogram_timing_results_list)
histogram_timing_results_list = []
counter = 0
for hist_result in array_hist_result:
    string="hist_timing_results/{}.csv".format(counter)
    f = open(string,"w+")
    for line in hist_result:
        f.write(line)
    f.close()
    counter+=1

# Now let's import the csv files into a dataframe. 

In [5]:
result_files = sorted(glob.glob(results_path),key=lambda x: int(x.split('.')[0][15:]))
frames = []
helper = []
counter = 0
for file in result_files:
    temp_df = pa.read_csv(str(file),'\t')
    helper.append([temp_df.iloc[0]['Test type'],counter])
    frames.append(temp_df)
    counter+=1

result_df = pa.concat(frames,sort=False)



hist_result_files = sorted(glob.glob(timing_histogram_path),key=lambda x: int(x.split('.')[0][20:]))
frames = []
counter = 0
for file in hist_result_files:
    temp_df = pa.read_csv(str(file),'\t')
 #   helper.append([temp_df.iloc[0]['Test type'],counter])
    frames.append(temp_df)
    counter+=1

hist_result_df = pa.concat(frames,sort=False)
hist_result_df=hist_result_df.astype({'No_of_executes': 'int32'})


#CREATE new SENS ds
sens_with_err = result_df.reset_index().drop(columns=['index'])
for index, row in sens_with_err.iterrows():
    sens_with_err.loc[index,'Internal run no'] = index%9
sens_with_err = sens_with_err.astype({'Internal run no': 'int'})

temp_sens = sens_with_err.groupby(['Test type','Parameter run no','Internal run no','Supplied parameters']).sum().reset_index().rename(columns={"Duration": "Query execution duration"})
temp_hist_sens = hist_result_df.groupby(['Test type','Parameter run no','Supplied parameters']).sum().reset_index().rename(columns={"Duration": "Histogram execution duration"})

another_temp_sens = pa.merge(temp_sens,temp_hist_sens,how='outer',on=['Test type','Parameter run no']).fillna(0)
another_temp_sens['Query and histogram execution duration'] = another_temp_sens['Query execution duration'] + another_temp_sens['Histogram execution duration']
another_temp_sens = another_temp_sens.drop(columns = ['Histogram execution duration','Number of Inserts','Returned rows','No_of_executes','Supplied parameters_y']).set_index(['Test type','Parameter run no','Internal run no','Supplied parameters_x'])

almost_done_df = pa.DataFrame(another_temp_sens,index = another_temp_sens.index).stack().to_frame().reset_index()
almost_done_df.columns = ['Test type','Parameter run no','Internal run no','Supplied parameters','Duration type','Duration']
almost_done_df.head(500)

temp_3 = almost_done_df
temp_3 = temp_3.loc[temp_3['Test type'].isin(['No histogram ','Perfect histogram ','Stale histogram ']) & temp_3['Duration type'].isin(['Query execution duration'])]
temp_4 = pa.merge(almost_done_df,temp_3, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
condensed_sens_with_err = temp_4
condensed_sens_with_err.drop_duplicates(keep = 'first', inplace = True) 

In [6]:

#Export the query_plans
array_query_plan = np.array(query_plan_list)
query_plan_list=[]
counter = 0
for plan_set in array_query_plan:
    string="query_plans/plan_{}_{}.txt".format(counter,helper[counter][0].strip())
    f = open(string,"w+")
    for plan in plan_set:
        f.write(plan)
    f.close()
    counter+=1

In [7]:
#Create the sensitivity DF
sensitivity_df = result_df.groupby(['Test type','Supplied parameters','Parameter run no']).sum().reset_index()
sensitivity_df = sensitivity_df[['Test type','Supplied parameters','Parameter run no','Duration']]
sensitivity_df = sensitivity_df.rename(columns={"Duration": "Query execution duration"})
sensitivity_df_hist_data = hist_result_df.groupby(['Test type','Supplied parameters','Parameter run no']).sum().reset_index().rename(columns={"Duration": "Histogram execution duration"})
temp = pa.merge(sensitivity_df,sensitivity_df_hist_data,how='outer',on=['Test type','Supplied parameters','Parameter run no'])
temp = temp.fillna(0)
temp['Query and histogram execution duration'] = temp['Query execution duration'] + temp['Histogram execution duration']
df_5 = temp
temp = temp.drop(columns = ['Histogram execution duration'])
temp = temp.set_index(['Test type','Supplied parameters','Parameter run no'])
sensitivity_df = pa.DataFrame(temp,index = temp.index).stack()
#new_df = new_df.columns(['Test type'])
sensitivity_df = sensitivity_df.to_frame()
sensitivity_df =sensitivity_df.reset_index()
sensitivity_df.columns = ['Test type','Supplied parameters','Parameter run no','Duration type','Duration']

#Splitting the params for the base classes test types
temp_data_df = sensitivity_df.loc[sensitivity_df['Test type'].isin(['No histogram ','Perfect histogram ','Stale histogram '])]
temp_data_df = temp_data_df[~temp_data_df['Duration type'].isin(['No_of_executes'])]
base_classes_data = temp_data_df[temp_data_df['Duration type'].isin(['Query and histogram execution duration'])].groupby(['Test type']).mean().reset_index()
base_classes_data['Duration type'] = 'Query and histogram execution duration'
base_classes_data['Supplied parameters']= 'Not applicable'
base_classes_data['Parameter run no']= 'Not applicable'
temp_data_df = temp_data_df.set_index(['Test type','Duration type','Duration','Parameter run no'])
temp_data_df = pa.DataFrame(temp_data_df['Supplied parameters'].str.split(' ').tolist(),index = temp_data_df.index).stack()
temp_data_df = temp_data_df.to_frame()
temp_data_df = temp_data_df.reset_index()
temp_data_df = temp_data_df.drop(columns='level_4')
temp_data_df.columns=['Test type','Duration type','Duration','Parameter run no','Supplied parameters']
sensitivity_df = sensitivity_df.loc[~sensitivity_df['Test type'].isin(['No histogram ','Perfect histogram ','Stale histogram '])]
sensitivity_df = sensitivity_df.append(temp_data_df, ignore_index=True,sort=False)
sensitivity_df = sensitivity_df.append(base_classes_data, ignore_index=True,sort=False)

#Before we remove some rows, let's save this to another df
full_sensitivity_df = sensitivity_df

#Removing the query and histogram execution duration for the base classes since we don't want that to show up in our plots. 
temp_3 = sensitivity_df
temp_3 = temp_3.loc[temp_3['Test type'].isin(['No histogram ','Perfect histogram ','Stale histogram ']) & temp_3['Duration type'].isin(['Query execution duration'])]
temp_4 = pa.merge(sensitivity_df,temp_3, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
sensitivity_df = temp_4
sensitivity_df.drop_duplicates(keep = 'first', inplace = True) 

# Format the queries to be q1 q2 etc. add a result_ratio column and a formatted query column

In [8]:
i=1
replace_dictionary = {}
unique_queries = result_df.Query.unique().tolist()
for query in unique_queries:
    name = 'q{}'.format(i)
    replace_dictionary[query]=name
    i+=1

result_df['Short_Query']=result_df['Query'].replace(replace_dictionary)

partition_size = min(result_df["Number of Inserts"].unique().tolist())
result_df['Result_ratio']=result_df['Returned rows']/partition_size

queries = result_df['Query'].tolist()
short_query_format=[]
for query in queries:
    no_of_joins = query.count('join')
    where_predicate_value = query.split("test.measurement.msm_value",1)[1]
    short_query_format.append('Query with '+str(no_of_joins)+' joins and WHERE MSM_VALUE' +where_predicate_value)
result_df['Short_query_format']=short_query_format




In [9]:
unformatted_plan_files = glob.glob(plans_path)
for file in unformatted_plan_files:
    output_file_name=file+".tmp"
    with open(file, 'r') as f, open(output_file_name, 'w+') as fo:
        for line in f:
            formatted_line = line.replace('explain format = tree ', '').replace(";","") 
           # print(formatted_line) 
            for query,key in replace_dictionary.items():
                if query==formatted_line:
                    fo.write(key)
                    break
            fo.write(line)
    os.remove(file)

## Prelim-results, don't run unless running prelim tests

In [ ]:
#ax=sns.lineplot(x="Number of Inserts", y="Duration",
#            hue="Test type", markers=True,
#            data=result_df[result_df.Short_Query.isin(['q2'])])
#q2_prelim_line_plot = plt.gcf()
#q2_prelim_line_plot.savefig("/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/plots/prelim/q2_prelim_line_plot.png",dpi=360)
#plt.show()

## Standard results

In [ ]:
base_rules = ['No histogram ','Perfect histogram ','Stale histogram ']
for rule in result_df['Test type'].unique().tolist():
    if rule not in base_rules:
        base_rules.append(rule)
        temp = result_df.set_index('Test type')
        temp = temp.loc[base_rules]
        temp = temp.reset_index()
        for params in temp['Parameter run no'].unique().tolist():
            for query in temp.Short_Query.unique().tolist():
                fig = plt.figure(figsize=(13,9))
                gs = gridspec.GridSpec(nrows=4, 
                                       ncols=1, 
                                       figure=fig, 
                                       height_ratios=[1, 1, 1, 1],
                                       wspace=0.3,
                                       hspace=0.3)

                ax1 = fig.add_subplot(gs[1:3,0])
                sns.lineplot(x="Number of Inserts", y="Duration",
                             hue="Test type", style="Test type",markers = True,dashes = True,
                             data=temp[temp.Short_Query.isin([query])&temp['Parameter run no'].isin([params])],ax=ax1)
                xlabels = ['{:,.0f}'.format(x) + 'K' for x in ax1.get_xticks()/1000]
                ax1.set_xticklabels(xlabels)
                ax1.set_xlabel("Number of updates")
                ax1.set_ylabel("Duration (s)")
               # ax1.set_title(str(result_df['Short_query_format'][result_df.Short_Query.isin([query])].iloc[0]))

                ax2 = fig.add_subplot(gs[0, 0])

                all_hist = result_df[result_df["Test type"].str.contains('Perfect histogram')]
                index_position = all_hist[all_hist.Short_Query.isin([query])].index[0]
                final = all_hist[all_hist.Short_Query.isin([query])].loc[index_position]
                sns.barplot(x="Number of Inserts", y="Result_ratio",data=final ,ax=ax2, color=sns.xkcd_rgb["denim blue"])


                ax2.spines['right'].set_visible(False)
                ax2.spines['top'].set_visible(False)
                #ax2.xaxis.set_major_locator(ax1.xaxis.get_major_locator())
                #ax2.set_xticklabels(ax1.get_xticklabels())
                ax2.set_xticklabels("")
                ax2.margins(x=ax1.margins()[0]-0.01)
                ax2.set_xlabel("")
                ax2.set_ylabel("Result size ratio")
              #  ax2.set_title(str(result_df['Short_query_format'][result_df.Short_Query.isin([query])].iloc[0]))

                #fig.suptitle(str(result_df['Short_query_format'][result_df.Short_Query.isin([query])].iloc[0]),y=0.9)

                line_plot = plt.gcf()
                path = "/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/eval_plots/line_plots/all_q_plots/"
                name = str(temp['Short_query_format'][temp.Short_Query.isin([query])].iloc[0])+str(base_rules)+str(params)+".png"
                line_plot.savefig(path+name,dpi=360,bbox_inches='tight')
                plt.close()
        base_rules.pop()
    if rule == 'No histogram ':
        temp = result_df.set_index('Test type')
        temp = temp.loc[base_rules]
        temp = temp.reset_index()
        for params in temp['Parameter run no'].unique().tolist():
            for query in temp.Short_Query.unique().tolist():
                fig = plt.figure(figsize=(13,9))
                gs = gridspec.GridSpec(nrows=4, 
                                       ncols=1, 
                                       figure=fig, 
                                       height_ratios=[1, 1, 1, 1],
                                       wspace=0.3,
                                       hspace=0.3)

                ax1 = fig.add_subplot(gs[1:3,0])
                sns.lineplot(x="Number of Inserts", y="Duration",
                             hue="Test type", style="Test type",markers = True,dashes = True,
                             data=temp[temp.Short_Query.isin([query])&temp['Parameter run no'].isin([params])],ax=ax1)
                xlabels = ['{:,.0f}'.format(x) + 'K' for x in ax1.get_xticks()/1000]
                ax1.set_xticklabels(xlabels)
                ax1.set_xlabel("Number of updates")
                ax1.set_ylabel("Duration (s)")
               # ax1.set_title(str(result_df['Short_query_format'][result_df.Short_Query.isin([query])].iloc[0]))

                ax2 = fig.add_subplot(gs[0, 0])

                all_hist = result_df[result_df["Test type"].str.contains('Perfect histogram')]
                index_position = all_hist[all_hist.Short_Query.isin([query])].index[0]
                final = all_hist[all_hist.Short_Query.isin([query])].loc[index_position]
                sns.barplot(x="Number of Inserts", y="Result_ratio",data=final ,ax=ax2, color=sns.xkcd_rgb["denim blue"])


                ax2.spines['right'].set_visible(False)
                ax2.spines['top'].set_visible(False)
                #ax2.xaxis.set_major_locator(ax1.xaxis.get_major_locator())
                #ax2.set_xticklabels(ax1.get_xticklabels())
                ax2.set_xticklabels("")
                ax2.margins(x=ax1.margins()[0]-0.01)
                ax2.set_xlabel("")
                ax2.set_ylabel("Result size ratio")
              #  ax2.set_title(str(result_df['Short_query_format'][result_df.Short_Query.isin([query])].iloc[0]))

                #fig.suptitle(str(result_df['Short_query_format'][result_df.Short_Query.isin([query])].iloc[0]),y=0.9)

                line_plot = plt.gcf()
                path = "/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/eval_plots/line_plots/all_q_plots/"
                name = str(temp['Short_query_format'][temp.Short_Query.isin([query])].iloc[0])+str(base_rules)+str(params)+".png"
                line_plot.savefig(path+name,dpi=360,bbox_inches='tight')
                plt.close()
    
fig = plt.figure(figsize=(9,6))
ax3=sns.barplot(x="Test type", y="Duration",hue="Test type", data=hist_result_df, estimator=sum,ci=None,dodge = False,palette = sns.color_palette("Paired", 9))
ax3.set_xticklabels("")
ax3.set_xlabel("")
ax3.set_ylabel("Duration (s)")
hist_timing_plt = plt.gcf()
hist_timing_plt.savefig("/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/eval_plots/histogram_timing.png",dpi=360,bbox_inches='tight')

# One off plots#

In [11]:
rules = ['No histogram ','Perfect histogram ','Stale histogram ']
rule_to_visit = 'Plugin rule 2'
rules.append(rule_to_visit)
params = '7'
temp = result_df.set_index('Test type')
temp = temp.loc[rules]
temp = temp.reset_index()
blah = temp
for query in temp.Short_Query.unique().tolist():
    fig = plt.figure(figsize=(13,9))
    gs = gridspec.GridSpec(nrows=4, 
                           ncols=1, 
                           figure=fig, 
                           height_ratios=[1, 1, 1, 1],
                           wspace=0.3,
                           hspace=0.3)

    ax1 = fig.add_subplot(gs[1:3,0])
    sns.lineplot(x="Number of Inserts", y="Duration",
                 hue="Test type", style="Test type",markers = True,dashes = True,
                 data=temp[temp.Short_Query.isin([query])&temp['Parameter run no'].isin([params])],ax=ax1)
    xlabels = ['{:,.0f}'.format(x) + 'K' for x in ax1.get_xticks()/1000]
    ax1.set_xticklabels(xlabels)
    ax1.set_xlabel("Number of updates")
    ax1.set_ylabel("Duration (s)")
    # ax1.set_title(str(result_df['Short_query_format'][result_df.Short_Query.isin([query])].iloc[0]))

    ax2 = fig.add_subplot(gs[0, 0])

    all_hist = result_df[result_df["Test type"].str.contains('Perfect histogram')]
    index_position = all_hist[all_hist.Short_Query.isin([query])].index[0]
    final = all_hist[all_hist.Short_Query.isin([query])].loc[index_position]
    sns.barplot(x="Number of Inserts", y="Result_ratio",data=final ,ax=ax2, color=sns.xkcd_rgb["denim blue"])


    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    #ax2.xaxis.set_major_locator(ax1.xaxis.get_major_locator())
    #ax2.set_xticklabels(ax1.get_xticklabels())
    ax2.set_xticklabels("")
    ax2.margins(x=ax1.margins()[0]-0.01)
    ax2.set_xlabel("")
    ax2.set_ylabel("Result size ratio")
    #  ax2.set_title(str(result_df['Short_query_format'][result_df.Short_Query.isin([query])].iloc[0]))

    #fig.suptitle(str(result_df['Short_query_format'][result_df.Short_Query.isin([query])].iloc[0]),y=0.9)


    line_plot = plt.gcf()

    path = "/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/eval_plots/line_plots/"+str(rule_to_visit)+"/"
    name = str(temp['Short_query_format'][temp.Short_Query.isin([query])].iloc[0])+str(rules)+str(params)+".png"#+str(temp['Supplied parameters'][temp.Short_Query.isin([query]) & temp['Test type'].isin([rule_to_visit])& temp['Parameter run no'].isin([params])].iloc[0])+".png"
    line_plot.savefig(path+name,dpi=360,bbox_inches='tight')
    plt.close()

In [ ]:
blah['Supplied parameters'][blah.Short_Query.isin([query]) & blah['Test type'].isin([rule_to_visit]) & blah['Parameter run no'].isin([params])].iloc[0]

# Generate Q-ratio data#

In [10]:
q_ratio_df=df_5.loc[df_5['Test type'].isin(['No histogram ','Perfect histogram ','Stale histogram ','Plugin rule 0'])].groupby(['Test type']).mean().reset_index()
q_ratio_df['Supplied parameters'] = 'Not applicable'
q_ratio_df = q_ratio_df.append(df_5.loc[~df_5['Test type'].isin(['No histogram ','Perfect histogram ','Stale histogram ','Plugin rule 0'])],ignore_index=True,sort=False)
q_ratio_df = q_ratio_df[['Test type','Query and histogram execution duration','Query execution duration','Histogram execution duration','No_of_executes','Supplied parameters']]
q_ratio_df=q_ratio_df.rename(columns={"Query and histogram execution duration": "Total duration", "No_of_executes": "Number of updates","Histogram execution duration":"Histogram duration","Query execution duration":"Query duration"})
q_ratio_df['Q-ratio']=q_ratio_df['Total duration'].apply(lambda x: x/q_ratio_df['Total duration'].loc[q_ratio_df['Test type']=='Perfect histogram '], 0)
q_ratio_df = q_ratio_df.sort_values('Q-ratio').set_index('Test type').astype({'Number of updates': 'int'}).reset_index()
q_ratio_df = q_ratio_df[['Test type','Total duration','Q-ratio','Query duration','Histogram duration','Number of updates','Supplied parameters']]
#Formatting
q_ratio_df['Test type'] = q_ratio_df['Test type'].replace("Plugin rule 0", "Rule 0") 
q_ratio_df['Test type'] = q_ratio_df['Test type'].replace("Plugin rule 2", "Rule 2") 
q_ratio_df['Test type'] = q_ratio_df['Test type'].replace("Plugin rule 3", "Rule 3") 
q_ratio_df['Test type'] = q_ratio_df['Test type'].replace("Plugin rule 6", "Rule 6") 
q_ratio_df['Test type'] = q_ratio_df['Test type'].replace("Plugin rule 7", "Rule 7") 
q_ratio_df['Test type'] = q_ratio_df['Test type'].replace("Plugin rule 9", "Rule 9") 



#Generate table file
q_ratio_df = q_ratio_df.reset_index()

temp = q_ratio_df.loc[q_ratio_df.groupby('Test type')['Q-ratio'].idxmin()]
temp_max = q_ratio_df.loc[q_ratio_df.groupby('Test type')['Q-ratio'].idxmax()]
temp = temp.append(temp_max).drop_duplicates(keep = 'first', inplace = False) 

summarised_q_ratio = temp.sort_values('Q-ratio')
summarised_q_ratio = summarised_q_ratio[['Test type','Total duration','Q-ratio','Query duration','Histogram duration','Number of updates','Supplied parameters']].set_index('Test type')

f = open('/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/eval_plots/q-ratio_table.tex',"w+")
f.write("\\noindent\makebox[\\textwidth]{%\n")
f.write("\\begin{tabularx}{\\textwidth}{LLLLLLL}\\toprule\n")
f.close()
summarised_q_ratio.to_csv('/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/eval_plots/q-ratio_table.tex',sep='&',line_terminator = "\\\\\n",mode='a',float_format='%.3f')
f = open('/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/eval_plots/q-ratio_table.tex','a')
f.write ("\\bottomrule\n")
f.write ("\\end{tabularx}\n")
f.write("}")
f.close()




#Show data here as well
summarised_q_ratio.head(5000)

,Total duration,Q-ratio,Query duration,Histogram duration,Number of updates,Supplied parameters
Test type,,,,,,
Rule 6,818.108483,0.998862,807.524671,10.583812,99,0.75
Perfect histogram,819.040614,1.000000,815.058959,3.981656,30,Not applicable
Rule 3,819.233852,1.000236,811.451387,7.782465,59,400000 2 10
Rule 2,826.609471,1.009241,818.820994,7.788477,59,200000
Rule 7,830.149213,1.013563,826.309386,3.839827,29,1
Rule 3,850.546712,1.038467,815.728716,34.817996,264,400000 8 10
Rule 0,912.709076,1.114364,912.709076,0.000000,0,Not applicable
No histogram,914.575439,1.116642,914.575439,0.000000,0,Not applicable
Rule 2,970.231563,1.184595,814.816326,155.415237,1180,10000


# Generate sensitivity plots for single parameter rules#

In [111]:
base_rules = ['No histogram ','Perfect histogram ','Stale histogram ']
rules_to_avoid_for_now = ['Plugin rule 9','Plugin rule 3']
for rule in condensed_sens_with_err['Test type'].unique().tolist():
    if rule not in base_rules and rule not in rules_to_avoid_for_now:
        current_rules=base_rules.copy()
        current_rules.append(rule)
        temp = condensed_sens_with_err.set_index('Test type')
        temp = temp.loc[current_rules]
        temp = temp[~temp['Supplied parameters'].isin(['Not applicable'])] 
        run_no_finder = temp.groupby('Test type')
        run_no_finder = run_no_finder.apply(lambda x: x['Parameter run no'].unique())
        unique_run_no_we_want_to_use = run_no_finder.loc[rule].tolist()
        param_list_finder = temp.groupby('Test type')
        param_list_finder = param_list_finder.apply(lambda x: x['Supplied parameters'].unique())
        unique_params_we_want_to_use = param_list_finder.loc[rule].tolist()
        temp = temp[temp['Parameter run no'].isin(unique_run_no_we_want_to_use)]
        rule_df = temp.loc[rule].reset_index()#.set_index(['Parameter run no'])
        base_df = temp.loc[base_rules].reset_index().set_index(['Parameter run no'])
        base_df['Supplied parameters'] = np.NaN
        for param_run_no in unique_run_no_we_want_to_use:
            base_df.loc[param_run_no,'Supplied parameters'] = unique_params_we_want_to_use[param_run_no-1]
        data_df = rule_df.append(base_df).astype({'Supplied parameters': 'float'})
        fig = plt.figure(figsize=(13,9))
        gs = gridspec.GridSpec(nrows=4, 
                           ncols=1, 
                           figure=fig, 
                           height_ratios=[1, 1, 1, 1],
                           wspace=0.3,
                           hspace=0.3)
        ax1 = fig.add_subplot(gs[1:3,0])
        g=sns.lineplot(x="Supplied parameters", y="Duration",
                     markers=True, hue = 'Test type', style = 'Duration type',ci=99,
                     data=data_df,ax=ax1)
        ax1.set_ylabel("Duration (s)")
        if rule == 'Plugin rule 0':
            plt.xscale("log")
            ax1.set_xticklabels("")
            ax1.set_xlabel("Paramter values - not applicable")  
        if rule == 'Plugin rule 2':
            plt.xscale("log")
            ax1.set_xlabel("Parameter values for n, \"Number of rows between updates\"")
            xlabels = ['{:,.0f}'.format(x) + 'K' for x in ax1.get_xticks()/1000]
            ax1.set_xticklabels(xlabels)
        if rule == 'Plugin rule 6':
            plt.xscale("linear")
            ax1.set_xlabel("Parameter values for r, \"Ratio of change before updates\"")
        if rule == 'Plugin rule 7':
            plt.xscale("linear")
            ax1.set_xlabel("Parameter values for w_, \"Update outside histogram range\"")
        g.legend_.remove()
        sensitivity_plot = plt.gcf()
        path = "/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/eval_plots/sensitivity_plots/"
        name = str(current_rules)+".png"
        sensitivity_plot.savefig(path+name,dpi=360,bbox_inches='tight')
        current_rules.pop
        plt.close()

        if rule == "Plugin rule 0":
            current_rules=base_rules.copy()
            current_rules.append(rule)
            temp = condensed_sens_with_err.set_index('Test type')
            temp = temp.loc[current_rules]
            temp = temp[~temp['Supplied parameters'].isin(['Not applicable'])] 
            run_no_finder = temp.groupby('Test type')
            run_no_finder = run_no_finder.apply(lambda x: x['Parameter run no'].unique())
            unique_run_no_we_want_to_use = run_no_finder.loc[rule].tolist()
            param_list_finder = temp.groupby('Test type')
            param_list_finder = param_list_finder.apply(lambda x: x['Supplied parameters'].unique())
            unique_params_we_want_to_use = param_list_finder.loc[rule].tolist()
            temp = temp[temp['Parameter run no'].isin(unique_run_no_we_want_to_use)]
            rule_df = temp.loc[rule].reset_index()#.set_index(['Parameter run no'])
            base_df = temp.loc[base_rules].reset_index().set_index(['Parameter run no'])
            base_df['Supplied parameters'] = np.NaN
            for param_run_no in unique_run_no_we_want_to_use:
                base_df.loc[param_run_no,'Supplied parameters'] = unique_params_we_want_to_use[param_run_no-1]
            data_df = rule_df.append(base_df).astype({'Supplied parameters': 'float'})
            data_df.head(200)
            fig = plt.figure(figsize=(13,9))
            gs = gridspec.GridSpec(nrows=4, 
                               ncols=1, 
                               figure=fig, 
                               height_ratios=[1, 1, 1, 1],
                               wspace=0.3,
                               hspace=0.3)
            ax1 = fig.add_subplot(gs[1:3,0])
            g=sns.lineplot(x="Supplied parameters", y="Duration",
                         markers=True, hue = 'Test type', style = 'Duration type',ci=99,
                         data=data_df,ax=ax1)
            plt.xscale("log")
            ax1.set_ylabel("Duration (s)")
            ax1.set_xlabel("Parameter values for a given rule X")
            ax1.set_xticklabels("")
           # handles, labels = ax1.get_legend_handles_labels()
           # ax1._legend.remove()
           # ax1.fig.legend(handles, labels, ncol=2, loc='upper center', 
                 #   bbox_to_anchor=(0.5, 1.15), frameon=False)
            #ax1.legend(frameon=False, loc='right', ncol=2,framealpha=1)
            handles, _ = g.get_legend_handles_labels()
            g.legend(handles,["Plugin rule","Plugin rule X","No histogram","Perfect histogram","Stale histogram","Duration type","Total duration","Query duration"],frameon=True,bbox_to_anchor=(0.35, 1.3),ncol=2, loc=2, borderaxespad=0.)
            #plt.legend(frameon=True,bbox_to_anchor=(0.25, 1.3),ncol=2, loc=2, borderaxespad=0.)
            sensitivity_plot = plt.gcf()
            path = "/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/eval_plots/sensitivity_plots/"
            name = "Special_explanation_plot"+".png"
            sensitivity_plot.savefig(path+name,dpi=360,bbox_inches='tight')
            plt.close()
            current_rules.pop()

In [113]:
data_df.head(100)

,Duration,Duration type,Internal run no,Parameter run no,Supplied parameters,Test type
0,91.837021,Query execution duration,0,1.0,1.0,Plugin rule 7
1,92.263668,Query and histogram execution duration,0,1.0,1.0,Plugin rule 7
2,91.836691,Query execution duration,1,1.0,1.0,Plugin rule 7
3,92.263338,Query and histogram execution duration,1,1.0,1.0,Plugin rule 7
4,91.798401,Query execution duration,2,1.0,1.0,Plugin rule 7
5,92.225048,Query and histogram execution duration,2,1.0,1.0,Plugin rule 7
6,91.751745,Query execution duration,3,1.0,1.0,Plugin rule 7
7,92.178392,Query and histogram execution duration,3,1.0,1.0,Plugin rule 7
8,91.787155,Query execution duration,4,1.0,1.0,Plugin rule 7
9,92.213802,Query and histogram execution duration,4,1.0,1.0,Plugin rule 7


# Then do the multiparam plots#

In [ ]:

cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)
for rule in sensitivity_df['Test type'].unique().tolist():
    if rule in rules_to_avoid_for_now:
        base_rules.append(rule)
        temp = sensitivity_df.set_index('Test type').loc[base_rules].reset_index().set_index('Duration type').loc[['Query and histogram execution duration','Query execution duration']].reset_index()
        for params in temp['Parameter run no'].unique().tolist():
            holder = temp[temp['Parameter run no'].isin([params])]
            if rule == 'Plugin rule 9':
                holder = holder.set_index('Test type').loc[[rule]].reset_index().set_index(['Test type','Duration type','Duration'])
                rule_9_df = pa.DataFrame(holder['Supplied parameters'].str.split(' ').tolist(),index = holder.index).reset_index().drop_duplicates(keep = 'first', inplace = False).rename(columns={0: "Weight_1", 1: "Weight_2",2: "Weight_3"}) 
                rule_9_df=rule_9_df.astype({'Weight_1': 'float','Weight_2': 'float','Weight_3': 'float'})
                fig = plt.figure(figsize=(13,9))
                gs = gridspec.GridSpec(nrows=4, 
                                   ncols=1, 
                                   figure=fig, 
                                   height_ratios=[1, 1, 1, 1],
                                   wspace=0.3,
                                   hspace=0.3)
                ax1 = fig.add_subplot(gs[1:3,0])
                sns.lineplot(x="Weight_1", y="Duration",
                             markers=True, hue = 'Test type', style = 'Duration type',
                             data=rule_9_df.loc[rule_9_df['Weight_2']=="200"],ax=ax1)
                ax1.set_ylabel("Duration (s)")
            if rule == 'Plugin rule 3':
              #  fig, axs = plt.subplots(1, 2)
                #fig.subplots_adjust(hspace=0.4, wspace=0.4)
                holder = temp.set_index('Test type').loc[[rule]].reset_index().set_index(['Test type','Duration type','Duration'])
                rule_3_df = pa.DataFrame(holder['Supplied parameters'].str.split(' ').tolist(),index = holder.index).reset_index().drop_duplicates(keep = 'first', inplace = False).rename(columns={0: "Weight_1", 1: "Weight_2",2: "Weight_3"}) 
                rule_3_df=rule_3_df.astype({'Weight_1': 'float','Weight_2': 'float','Weight_3': 'float'})
                #ax1 = fig.add_subplot(1, 1.5, 1)
               # sns.lineplot(x="Weight_1", y="Duration",
              #               markers=True, hue = 'Test type', style = 'Duration type',
              #               data=rule_3_df.loc[rule_3_df['Weight_2']=="20"],ax=axs[0,0])
              #  ax1.set_ylabel("Duration (s)")
                fig = plt.figure(figsize=(13,9))
                sns.scatterplot(x="Weight_1", y="Weight_2",
                         hue = "Duration type",size="Duration",
                          sizes=(10, 200),
                         data=rule_3_df)#,ax=ax2)
            if rule == 'Plugin rule 7':
                holder = temp.set_index('Test type').loc[[rule]].reset_index().set_index(['Test type','Duration type','Duration'])
                rule_7_df = pa.DataFrame(holder['Supplied parameters'].str.split(' ').tolist(),index = holder.index).reset_index().drop_duplicates(keep = 'first', inplace = False).rename(columns={0: "Weight_1", 1: "Weight_2",2: "Weight_3"}) 
                rule_7_df=rule_7_df.astype({'Weight_1': 'float','Weight_2': 'float'})
        base_rules.pop()
rule_7_df.head(500)


In [ ]:
g = sns.FacetGrid(rule_3_df, 
                  col="Weight_1",            # TWO COLUMNS coz there're TWO "wine types"
                #  col_order=['red', 'white'], # -> Specify the labels
                  hue='Duration type',        # ADD COLOR
                 # hue_order=['low', 'medium', 'high'],
                  aspect=1.2, 
                  size=3.5)

g.map(plt.scatter,x = rule_3_df['Weight_2'], 
            y = rule_3_df['Duration'], 
            s = rule_3_df['Weight_3']*25, # <== 😀 Look here!
            alpha=0.4, 
            edgecolors='w')

plt.xlabel('Param 2')
plt.ylabel('Duration')
plt.title('Wine Alcohol Content - Fixed Acidity - Residual Sugar', y=1.05)
rule_3_df.head(500)

In [ ]:
g = sns.FacetGrid(rule_3_df, 
                  col="Duration type",            # 😀 TWO COLUMNS coz there're TWO "wine types"
                  #col_order=['Query duration', 'Total duration'], # -> Specify the labels
                  hue='Weight_1',        # ADD COLOR
                 # hue_order=['low', 'medium', 'high'],
                  aspect=1.2, 
                  height=3.5, 
                  palette=sns.light_palette('navy', 4)[1:])

g.map(plt.scatter, 
      x=rule_3_df["Weight_2"], # <== x-axis
      y=rule_3_df["Duration"],          # <== y-axis
      alpha=0.9, 
      edgecolor='white', linewidth=0.5, s = rule_3_df['Weight_3']*25, # <== 😀 Look here!
           )

fig = g.fig 
fig.subplots_adjust(top=0.8, wspace=0.3)
fig.suptitle('Wine Type - Alcohol - Quality - Acidity', fontsize=14)
g.add_legend(title='Param 1 value')

In [ ]:
sensitivity_df.head(500)

In [ ]:
rule = 'Plugin rule 7'
rules=['No histogram ','Plugin rule 7']

rule_df = sensitivity_df.set_index('Test type').loc[rule].reset_index().set_index('Duration type').loc[['Query and histogram execution duration','Query execution duration']].reset_index()
param_runs = rule_df['Parameter run no'].unique().tolist()
rule_df = rule_df.set_index(['Test type','Duration type','Duration','Parameter run no'])
if rule == 'Plugin rule 3':
    rule_3_df = pa.DataFrame(rule_df['Supplied parameters'].str.split(' ').tolist(),index = rule_df.index).reset_index().drop_duplicates(keep = 'first', inplace = False).rename(columns={0: "Weight_1", 1: "Weight_2",2: "Weight_3"}).astype({'Weight_1': 'float','Weight_2': 'float','Weight_3': 'float'})
    fig = plt.figure(figsize=(13,9))
    gs = gridspec.GridSpec(nrows=4, 
                       ncols=1, 
                       figure=fig, 
                       height_ratios=[1, 1, 1, 1],
                       wspace=0.3,
                       hspace=0.3)
    ax1 = fig.add_subplot(gs[1:3,0])
    sns.lineplot(x="Weight_2", y="Duration",
                 markers=True, hue = 'Test type', style = 'Duration type',
                 data=rule_3_df[rule_3_df['Weight_2']!=rule_3_df['Weight_3']],ax=ax1)
    ax1.set_ylabel("Duration (s)")
    plt.close()
    fig = plt.figure(figsize=(13,9))
    gs = gridspec.GridSpec(nrows=4, 
                       ncols=1, 
                       figure=fig, 
                       height_ratios=[1, 1, 1, 1],
                       wspace=0.3,
                       hspace=0.3)
    ax1 = fig.add_subplot(gs[1:3,0])
    sns.lineplot(x="Weight_3", y="Duration",
                 markers=True, hue = 'Test type', style = 'Duration type',
                 data=rule_3_df[rule_3_df['Weight_2']!=rule_3_df['Weight_3']],ax=ax1)
    ax1.set_ylabel("Duration (s)")
    plt.close()
    
if rule == 'Plugin rule 7':
    rule_7_df = pa.DataFrame(rule_df['Supplied parameters'].str.split(' ').tolist(),index = rule_df.index).reset_index().drop_duplicates(keep = 'first', inplace = False).rename(columns={0: "Weight_1", 1: "Weight_2"}).astype({'Weight_1': 'float','Weight_2': 'float'})
    fig = plt.figure(figsize=(13,9))
    gs = gridspec.GridSpec(nrows=4, 
                       ncols=1, 
                       figure=fig, 
                       height_ratios=[1, 1, 1, 1],
                       wspace=0.3,
                       hspace=0.3)
    ax1 = fig.add_subplot(gs[1:3,0])
    sns.lineplot(x="Weight_2", y="Duration",
                 markers=True, hue = 'Test type', style = 'Duration type',
                 data=rule_7_df[rule_7_df['Weight_1']!=rule_7_df['Weight_2']],ax=ax1)
    ax1.set_ylabel("Duration (s)")
#for params in temp['Supplied parameters'][temp['Test type'].isin([rule])].unique().tolist():
#    holder = temp
#    print (params)
    
 #   holder = temp[temp['Parameter run no'].isin([params])]
 #   if rule == 'Plugin rule 7':
 #       print(holder.head(500))
rule_7_df.head(500)

In [ ]:


rule_7_df = rule_7_df.append(sensitivity_df[sensitivity_df['Test type'].isin(['No histogram ','Stale histogram ','Perfect histogram ']) & sensitivity_df['Parameter run no'].isin(param_runs)])

rule_7_df.head(500)


In [ ]:
temp = sensitivity_df.set_index('Test type')
temp = temp.loc[['No histogram ','Perfect histogram ','Stale histogram ','Plugin rule 2']]
temp = temp[~temp['Supplied parameters'].isin(['Not applicable'])]
temp=temp.astype({'Supplied parameters': 'float'})
holder = temp.groupby('Test type')
holder = holder.apply(lambda x: x['Supplied parameters'].unique())
unique_params_we_want_to_use = holder.loc['Plugin rule 2'].tolist()
temp = temp.loc[temp['Supplied parameters'].isin(unique_params_we_want_to_use)].reset_index().set_index('Test type').loc[['No histogram ','Perfect histogram ','Stale histogram ']].reset_index()

temp.head(500)

# Average q-ratio, må kanskje endre slik at vi lager en q-ratio for hvert sett med parameter verdier , Ser ut som at denne kanskje er litt feil, se på q-ratio...#

In [ ]:
q_ratio_df = result_df.groupby(['Test type','Supplied parameters']).sum().reset_index()
q_ratio_df = q_ratio_df[['Test type','Duration','Supplied parameters']]
q_ratio_df = q_ratio_df.groupby(['Test type']).mean().reset_index()
q_ratio_df['Q-ratio']=q_ratio_df['Duration'].apply(lambda x: x/q_ratio_df['Duration'].loc[q_ratio_df['Test type']=='Perfect histogram '], 0)
q_ratio_df = q_ratio_df.sort_values('Q-ratio')
#q_ratio_df.head(100)

hist_result_df=hist_result_df.astype({'No_of_executes': 'int32'})
histogram_data = hist_result_df.groupby(['Test type','Supplied parameters'])['Duration','No_of_executes'].sum().reset_index()
histogram_data = histogram_data.groupby(['Test type']).mean().reset_index()
histogram_data=histogram_data.rename(columns={"Duration": "Time spent computing histograms (s)", "No_of_executes": "Number of histogram udpates/creates"})
#histogram_data.head(50)

merged_df = q_ratio_df.merge(histogram_data, how = 'outer', on = ['Test type'])
merged_df = merged_df.fillna(0)
merged_df = merged_df.astype({'Number of histogram udpates/creates':'int32'})
merged_df.head(50)

In [ ]:
##Working og gettign the data we want to create our standard query execution times plots for
xlabels = ['{:,.4f}'.format(float(re.sub("\D", "", str(x)))) for x in  ['r=0.1','r=0.2','r=0.4']]
a = [re.sub("\d", "", x) for x in ['r=0.1','r=0.2','r=0.4']]
print (xlabels)
print(ax1.get_xticklabels())

In [ ]:
x='aaa12333bb445bb54b5b52'
import re
re.sub("\D", "", "aas30dsa20")

In [ ]:
df_1 = sensitivity_df.loc[sensitivity_df['Test type'].isin(['Plugin rule 9'])]

df_1 = df_1.append([df_1]*2,ignore_index=True)
df_1.at[0,'Supplied parameters'] = 'bw=2 I=50000'
df_1.at[1,'Supplied parameters'] = 'bw=2 I=50000'
df_1.at[2,'Supplied parameters'] = 'bw=200000 I=5'
df_1.at[3,'Supplied parameters'] = 'bw=200000 I=5'
df_1.at[4,'Supplied parameters'] = 'bw=20 I=50000'
df_1.at[5,'Supplied parameters'] = 'bw=20 I=50000'
df_1.at[6,'Supplied parameters'] = 'bw=200000 I=500'
df_1.at[7,'Supplied parameters'] = 'bw=200000 I=500'
df_1.at[8,'Supplied parameters'] = 'bw=200000 I=50'
df_1.at[9,'Supplied parameters'] = 'bw=200000 I=50'
df_1.at[10,'Supplied parameters'] = 'bw=200000 I=5'
df_1.at[11,'Supplied parameters'] = 'bw=200000 I=5'
df_1=df_1.set_index(['Test type','Duration type','Duration'])

df_2 = pa.DataFrame(df_1['Supplied parameters'].str.split(' ',1).tolist(),index = df_1.index,columns=['param_1','param_2'])
df_1 = df_1.reset_index()
df_2=df_2.reset_index()
data_for_now = df_2.groupby(['param_1'])
data_for_now.head(500)

In [ ]:
fig = plt.figure(figsize=(13,9))
gs = gridspec.GridSpec(nrows=4, 
                   ncols=1, 
                   figure=fig, 
                   height_ratios=[1, 1, 1, 1],
                   wspace=0.3,
                   hspace=0.3)
ax1 = fig.add_subplot(gs[1:3,0])
sns.lineplot(x="param_1", y="Duration",
             markers=True, hue = 'param_2', style = 'Duration type',
             data=df_2,ax=ax1)
ax1.set_ylabel("Duration (s)")
#xlabels = ['{:,.4f}'.format(float(re.sub("\D", "", str(x)))) for x in ax1.get_xticks()]
#ax1.set_xticklabels(xlabels)
ax1.set_xlabel("Outside boundary weight parameter")

fig = plt.figure(figsize=(13,9))
gs = gridspec.GridSpec(nrows=4, 
                   ncols=1, 
                   figure=fig, 
                   height_ratios=[1, 1, 1, 1],
                   wspace=0.3,
                   hspace=0.3)
ax1 = fig.add_subplot(gs[1:3,0])
sns.lineplot(x="Supplied parameters", y="Duration",
             markers=True, hue = 'Test type', style = 'Duration type',
             data=df_1,ax=ax1)
ax1.set_ylabel("Duration (s)")
#xlabels = ['{:,.4f}'.format(float(re.sub("\D", "", str(x)))) for x in ax1.get_xticks()]
#ax1.set_xticklabels(xlabels)
ax1.set_xlabel("Supplied parameters")

In [ ]:
fig = plt.figure(figsize=(9,6))
ax3=sns.barplot(x="Test type", y="Duration",hue="Test type", data=hist_result_df[hist_result_df['Test type'].isin(['No histogram ','Perfect histogram ','Stale histogram '])], estimator=sum,ci=None,dodge = False )#,color=sns.xkcd_rgb["orange"])
ax3.set_xticklabels("")
ax3.set_xlabel("")
ax3.set_ylabel("Duration (s)")
hist_timing_plt = plt.gcf()
hist_timing_plt.savefig("/export/home/tmp/Dropbox/Apper/ShareLaTeX/Master/eval_plots/histogram_timing_base.png",dpi=360,bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(9,6))
ax3=sns.barplot(x="Test type", y="Duration",hue="Test type", data=hist_result_df, estimator=sum,ci=None,dodge = False )#,color=sns.xkcd_rgb["orange"])
ax3.set_xticklabels("")
ax3.set_xlabel("")
ax3.set_ylabel("Duration (s)")

In [ ]:
sns.palplot(sns.color_palette("Paired", 9))

In [ ]:
new_map = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628','#f781bf','#999999']
sns.palplot(sns.color_palette(new_map))

In [ ]:
d = {'Fixed data without Histograms': result_0_df, 'Fixed data with Histograms': result_1_df}
temp=pa.concat(d,axis=1)
temp=temp.transpose().drop(index='Average',level=1)
temp = temp.drop(index='Returned rows',level=1)
#temp = temp.drop(index='Query',level=1)
temp2=pa.concat([result_0_df,result_1_df],join='inner',axis=1)
temp2=temp2.drop(columns='Average')